# 🧠 Simple Neural Architectures (Minimal)

This notebook uses pre-computed embeddings + simple neural networks.
No transformer fine-tuning = No crashes!

In [2]:
# Crash prevention
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["OMP_NUM_THREADS"] = "2"

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from pathlib import Path

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cpu


In [3]:
# Load data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

print(f"Train: {len(train_df):,}")
print(f"Test: {len(test_df):,}")

Train: 2,029
Test: 54,059


In [6]:
# Load sentence encoder (pre-trained, no fine-tuning)
print("Loading sentence encoder...")
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode texts
print("Encoding train texts...")
train_embeddings = encoder.encode(train_df['body'].tolist(), show_progress_bar=True)

print("Encoding test texts...")
test_embeddings = encoder.encode(test_df['body'].tolist(), show_progress_bar=True)

print(f"\nEmbedding shape: {train_embeddings.shape}")

Loading sentence encoder...
Encoding train texts...


Batches: 100%|██████████| 64/64 [00:45<00:00,  1.41it/s]


Encoding test texts...


Batches:   3%|▎         | 48/1690 [01:05<37:17,  1.36s/it]


KeyboardInterrupt: 

In [ ]:
# Improved MLP Classifier
class BetterMLPClassifier(nn.Module):
    def __init__(self, input_dim=384):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LayerNorm(256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),

            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        return self.network(x).squeeze()

model = BetterMLPClassifier().to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


Model parameters: 132,353


In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(
    train_embeddings,
    train_df['rule_violation'].values,
    test_size=0.2,
    random_state=SEED,
    stratify=train_df['rule_violation']
)

print(f"Train: {len(X_train):,}")
print(f"Val: {len(X_val):,}")

Train: 1,623
Val: 406


In [ ]:
# Train
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

epochs = 6#10
batch_size = 64

model.train()
for epoch in range(epochs):
    total_loss = 0
    for i in range(0, len(X_train), batch_size):
        batch_X = torch.FloatTensor(X_train[i:i+batch_size]).to(device)
        batch_y = torch.FloatTensor(y_train[i:i+batch_size]).to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # Validate
    model.eval()
    with torch.no_grad():
        val_preds = []
        for i in range(0, len(X_val), batch_size):
            batch_X = torch.FloatTensor(X_val[i:i+batch_size]).to(device)
            outputs = model(batch_X)
            val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
    
    val_auc = roc_auc_score(y_val, val_preds)
    model.train()
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(X_train)*batch_size:.4f} - Val AUC: {val_auc:.4f}")

print(f"\n✅ Training complete!")

NameError: name 'model' is not defined

In [ ]:
# Predict on test
model.eval()
test_predictions = []

with torch.no_grad():
    for i in range(0, len(test_embeddings), batch_size):
        batch_X = torch.FloatTensor(test_embeddings[i:i+batch_size]).to(device)
        outputs = model(batch_X)
        test_predictions.extend(torch.sigmoid(outputs).cpu().numpy())

test_predictions = np.array(test_predictions)

print(f"Predictions: {len(test_predictions)}")
print(f"Min: {test_predictions.min():.4f}")
print(f"Max: {test_predictions.max():.4f}")
print(f"Mean: {test_predictions.mean():.4f}")

Predictions: 54059
Min: 0.0269
Max: 0.9681
Mean: 0.4184


In [ ]:
# Save submission
Path('outputs').mkdir(exist_ok=True)

submission = pd.DataFrame({
    'row_id': test_df['row_id'],
    'rule_violation': test_predictions
})

submission.to_csv('outputs/submission_nb2.csv', index=False)
print("✅ Saved to outputs/submission_nb2.csv")
print(submission.head())

✅ Saved to outputs/submission_nb2.csv
   row_id  rule_violation
0    2029        0.624840
1    2030        0.404855
2    2031        0.137449
3    2032        0.901082
4    2033        0.717101
